<a href="https://colab.research.google.com/github/dgutiluns/water-anomaly-ca/blob/main/joining_all_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor, IsolationForest
from google.colab import drive, files
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [75]:
# path_cleaned = '/content/drive/MyDrive/Year 3 Berk/Data Discovery Group Folder/Data/clean_water_table_for_weather_merge.csv'
# strct_smpl = pd.read_csv(path_cleaned)
# strct_smpl

#path_cleaned = '/content/drive/MyDrive/2025 fall/Data Discovery Group Folder/Data/clean_water_table_for_weather_merge.csv'
path_cleaned = '/content/drive/MyDrive/Year 3 Berk/Data Discovery Group Folder/Data/clean_water_table_for_weather_merge.csv'
strct_smpl = pd.read_csv(path_cleaned, parse_dates=['SampleDateTime'])
strct_smpl

,StationCode,COMID,SampleDateTime,Latitude,Longitude,Year,Month,DOY,Season,Result_ugL,...,prism_norm_8110.precip8110ws,prism_norm_8110.tmean8110ws,prism_norm_8110.tmax8110ws,prism_norm_8110.tmin8110ws,wetness.wetindexws,wetness.wetindexcat,runoff.runoffws,runoff.runoffcat,landfire_veg.pctnonagintrodmanagvegws,landfire_veg.pctnonagintrodmanagvegcat
0,HGA,20329198,2017-04-18,33.04581,-117.12715,2017,4,108,Spring,50.0,...,477.242082,16.407631,24.216835,8.593340,687.8284,714.2622,38.9634,39.3049,16.61,1.61
1,HGA,20329198,2017-04-18,33.04581,-117.12715,2017,4,108,Spring,50.0,...,477.242082,16.407631,24.216835,8.593340,687.8284,714.2622,38.9634,39.3049,16.61,1.61
2,HGA,20329198,2017-04-18,33.04581,-117.12715,2017,4,108,Spring,492.0,...,477.242082,16.407631,24.216835,8.593340,687.8284,714.2622,38.9634,39.3049,16.61,1.61
3,HGA,20329198,2017-04-18,33.04581,-117.12715,2017,4,108,Spring,662.0,...,477.242082,16.407631,24.216835,8.593340,687.8284,714.2622,38.9634,39.3049,16.61,1.61
4,HGA,20329198,2017-04-18,33.04581,-117.12715,2017,4,108,Spring,759.0,...,477.242082,16.407631,24.216835,8.593340,687.8284,714.2622,38.9634,39.3049,16.61,1.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3111,RBC02,22549103,2022-01-10 09:48:00,33.41544,-117.15199,2022,1,10,Winter,100.0,...,433.220501,17.602009,24.055591,11.144219,678.6893,688.0435,28.8988,28.6356,2.50,4.69
3112,RBC04,22549103,2022-01-10 10:26:00,33.41233,-117.15972,2022,1,10,Winter,1790000.0,...,433.220501,17.602009,24.055591,11.144219,678.6893,688.0435,28.8988,28.6356,2.50,4.69
3113,RBC04,22549103,2022-01-10 10:26:00,33.41233,-117.15972,2022,1,10,Winter,56.0,...,433.220501,17.602009,24.055591,11.144219,678.6893,688.0435,28.8988,28.6356,2.50,4.69
3114,RGT01,22548465,2022-01-10 10:55:00,33.41107,-117.18569,2022,1,10,Winter,1250000.0,...,426.989420,17.604599,23.618237,11.585818,617.7526,617.7526,27.0000,27.0000,2.69,2.69


In [76]:
na_counts = strct_smpl.isna().sum()
print(na_counts)

StationCode                               0
COMID                                     0
SampleDateTime                            0
Latitude                                  0
Longitude                                 0
                                         ..
wetness.wetindexcat                       0
runoff.runoffws                           0
runoff.runoffcat                          0
landfire_veg.pctnonagintrodmanagvegws     0
landfire_veg.pctnonagintrodmanagvegcat    0
Length: 137, dtype: int64


In [77]:
path_prism = '/content/drive/MyDrive/Year 3 Berk/Data Discovery Group Folder/Data/all_data.csv'
#path_prism = '/content/drive/MyDrive/2025 fall/Data Discovery Group Folder/Data/all_data.csv'
prism = pd.read_csv(path_prism)
prism = prism.iloc[:, :12]

/tmp/ipython-input-1812677865.py:3: DtypeWarning: Columns (0,4,12) have mixed types. Specify dtype option on import or set low_memory=False.
  prism = pd.read_csv(path_prism)


In [78]:
display(prism[["Longitude", "Latitude", "Date"]])
display(strct_smpl[["Longitude", "Latitude", "SampleDateTime"]])


,Longitude,Latitude,Date
0,-117.1271,33.0458,1/1/2017
1,-117.1271,33.0458,1/2/2017
2,-117.1271,33.0458,1/3/2017
3,-117.1271,33.0458,1/4/2017
4,-117.1271,33.0458,1/5/2017
...,...,...,...
1523580,-117.1834,33.4070,12/27/2024
1523581,-117.1834,33.4070,12/28/2024
1523582,-117.1834,33.4070,12/29/2024
1523583,-117.1834,33.4070,12/30/2024


,Longitude,Latitude,SampleDateTime
0,-117.12715,33.04581,2017-04-18
1,-117.12715,33.04581,2017-04-18
2,-117.12715,33.04581,2017-04-18
3,-117.12715,33.04581,2017-04-18
4,-117.12715,33.04581,2017-04-18
...,...,...,...
3111,-117.15199,33.41544,2022-01-10 09:48:00
3112,-117.15972,33.41233,2022-01-10 10:26:00
3113,-117.15972,33.41233,2022-01-10 10:26:00
3114,-117.18569,33.41107,2022-01-10 10:55:00


In [79]:
strct_smpl["DOY"]

,DOY
0,108
1,108
2,108
3,108
4,108
...,...
3111,10
3112,10
3113,10
3114,10


In [80]:
prism[(prism["Longitude"] == -121.8007	) & (prism["Latitude"] == 36.8905) & (prism["Date"] == "4/27/2022")]

,Name,Longitude,Latitude,Elevation (ft),Date,ppt (inches),tmin (degrees F),tmean (degrees F),tmax (degrees F),tdmean (degrees F),vpdmin (hPa),vpdmax (hPa)
958083,Location 17,-121.8007,36.8905,13.0,4/27/2022,0.0,44.9,52.8,60.7,47.3,0.25,6.38


In [49]:
# @title
import pandas as pd
#JOIN VERSION 1
# NOT USING
# Always work on copies
strct_smpl = strct_smpl.copy()
prism = prism.copy()

# 1) Parse to datetime (robust)
strct_smpl["SampleDateTime"] = pd.to_datetime(
    strct_smpl["SampleDateTime"], errors="coerce"
)

#strct_smpl["SampleDateTime"] = strct_smpl["SampleDateTime"].str[:10]


# PRISM sometimes has mixed/dirty strings — extract the first MM/DD/YYYY pattern and parse it
prism_date_str = (
    prism["Date"].astype(str).str.strip()
         .str.extract(r'(\d{1,2}/\d{1,2}/\d{4})', expand=False)
)
# parse dates
prism["Date"] = pd.to_datetime(prism["Date"].astype(str).str.strip(),
                               format="%m/%d/%Y", errors="coerce")
strct_smpl["date_key"] = strct_smpl["SampleDateTime"].dt.normalize()
prism["date_key"]      = prism["Date"].dt.normalize()

# coord keys
strct_smpl["lat_key"] = pd.to_numeric(strct_smpl["Latitude"],  errors="coerce").round(3)
strct_smpl["lon_key"] = pd.to_numeric(strct_smpl["Longitude"], errors="coerce").round(3)
prism["lat_key"]      = pd.to_numeric(prism["Latitude"],       errors="coerce").round(3)
prism["lon_key"]      = pd.to_numeric(prism["Longitude"],      errors="coerce").round(3)

# --- 3) Keep/rename PRISM weather columns ---
weather_cols = [
    "ppt (inches)","tmin (degrees F)","tmean (degrees F)","tmax (degrees F)",
    "tdmean (degrees F)","vpdmin (hPa)","vpdmax (hPa)","Elevation (ft)"
]
prism_ren = prism[["date_key","lat_key","lon_key"] + weather_cols].rename(
    columns={
        "ppt (inches)": "prism_ppt_in",
        "tmin (degrees F)": "prism_tmin_F",
        "tmean (degrees F)": "prism_tmean_F",
        "tmax (degrees F)": "prism_tmax_F",
        "tdmean (degrees F)": "prism_tdmean_F",
        "vpdmin (hPa)": "prism_vpdmin_hPa",
        "vpdmax (hPa)": "prism_vpdmax_hPa",
        "Elevation (ft)": "prism_elev_ft",
    }
)

# If multiple PRISM rows share the same (lat, lon, date), collapse (mean keeps it simple)
prism_ren = (
    prism_ren
    .groupby(["date_key","lat_key","lon_key"], as_index=False)
    .mean(numeric_only=True)
)

# --- 4) Left-merge onto strct_smpl ---
merge_keys = ["date_key","lat_key","lon_key"]
strct_with_weather = strct_smpl.merge(prism_ren, on=merge_keys, how="left")

# --- 5) Clean up helper keys if you don’t need them downstream ---
strct_with_weather.drop(columns=["date_key","lat_key","lon_key"], inplace=True)

# Quick sanity checks
print("Rows in strct_smpl:", len(strct_smpl))
print("Rows after merge  :", len(strct_with_weather))
added = [c for c in strct_with_weather.columns if c.startswith("prism_")]
print("Added PRISM cols  :", added)

# Peek
strct_with_weather[["Longitude","Latitude","SampleDateTime"] + added]


KeyboardInterrupt: 

rows: 3116
rows with any PRISM value: 3116


In [81]:
import pandas as pd
import numpy as np

# --- copies ---
strct = strct_smpl.copy()
wx    = prism.copy()

# 1) PRISM: parse known M/D/YYYY and make day key
wx["Date"]    = pd.to_datetime(wx["Date"].astype(str).str.strip(),
                               format="%m/%d/%Y", errors="coerce")
wx["date_key"] = wx["Date"].dt.normalize()

# 2) Samples: ensure datetime and REPAIR NaT
strct["SampleDateTime"] = pd.to_datetime(strct["SampleDateTime"], errors="coerce")
nat = strct["SampleDateTime"].isna()
print("NaT in SampleDateTime BEFORE repair:", int(nat.sum()))

# 2a) Try to rebuild from Year + DOY (day-of-year)
if nat.any() and {"Year","DOY"}.issubset(strct.columns):
    y   = pd.to_numeric(strct.loc[nat, "Year"], errors="coerce").astype("Int64")
    doy = pd.to_numeric(strct.loc[nat, "DOY"],  errors="coerce").astype("Int64")
    rebuilt = (
        pd.to_datetime(y.astype(str), format="%Y", errors="coerce")
        + pd.to_timedelta(doy.sub(1), unit="D")
    )
    strct.loc[nat, "SampleDateTime"] = rebuilt
    nat = strct["SampleDateTime"].isna()

# 2b) If still NaT and you have SampleDate/CollectionTime, rebuild timestamp
if nat.any() and {"SampleDate","CollectionTime"}.issubset(strct.columns):
    print("went into 2b")
    d = pd.to_datetime(strct.loc[nat, "SampleDate"], errors="coerce")
    tt = strct.loc[nat, "CollectionTime"].astype(str).str.extract(r"(\d{1,2}):(\d{2})(?::(\d{2}))?")
    hh = pd.to_numeric(tt[0], errors="coerce").fillna(0)
    mm = pd.to_numeric(tt[1], errors="coerce").fillna(0)
    ss = pd.to_numeric(tt[2], errors="coerce").fillna(0)
    strct.loc[nat, "SampleDateTime"] = d + pd.to_timedelta(hh, "h") + pd.to_timedelta(mm, "m") + pd.to_timedelta(ss, "s")
    nat = strct["SampleDateTime"].isna()

print("NaT in SampleDateTime AFTER  repair:", int(nat.sum()))

# 3) Build day key and coord keys
strct["date_key"] = strct["SampleDateTime"].dt.normalize()
strct["lat_key"]  = pd.to_numeric(strct["Latitude"],  errors="coerce").round(3)
strct["lon_key"]  = pd.to_numeric(strct["Longitude"], errors="coerce").round(3)
wx["lat_key"]     = pd.to_numeric(wx["Latitude"],     errors="coerce").round(3)
wx["lon_key"]     = pd.to_numeric(wx["Longitude"],    errors="coerce").round(3)

# 4) Prepare unique PRISM (one row per date/lat/lon)
weather_cols = [
    "ppt (inches)","tmin (degrees F)","tmean (degrees F)","tmax (degrees F)",
    "tdmean (degrees F)","vpdmin (hPa)","vpdmax (hPa)","Elevation (ft)"
]
wxu = (
    wx[["date_key","lat_key","lon_key"] + weather_cols]
      .dropna(subset=["date_key","lat_key","lon_key"])
      .rename(columns={
          "ppt (inches)":"prism_ppt_in",
          "tmin (degrees F)":"prism_tmin_F",
          "tmean (degrees F)":"prism_tmean_F",
          "tmax (degrees F)":"prism_tmax_F",
          "tdmean (degrees F)":"prism_tdmean_F",
          "vpdmin (hPa)":"prism_vpdmin_hPa",
          "vpdmax (hPa)":"prism_vpdmax_hPa",
          "Elevation (ft)":"prism_elev_ft",
      })
      .groupby(["date_key","lat_key","lon_key"], as_index=False)
      .mean(numeric_only=True)
)

# 5) LEFT merge: keeps all sample rows
merge_keys = ["date_key","lat_key","lon_key"]
strct_with_weather = strct.merge(wxu, on=merge_keys, how="left").drop(columns=merge_keys)

print("Rows in samples:", len(strct))
print("Rows after merge:", len(strct_with_weather))
print("Rows with any PRISM value:",
      strct_with_weather.filter(like="prism_").notna().any(axis=1).sum())


NaT in SampleDateTime BEFORE repair: 2387
NaT in SampleDateTime AFTER  repair: 0
Rows in samples: 3116
Rows after merge: 3116
Rows with any PRISM value: 3116


In [82]:
strct_with_weather

,StationCode,COMID,SampleDateTime,Latitude,Longitude,Year,Month,DOY,Season,Result_ugL,...,landfire_veg.pctnonagintrodmanagvegws,landfire_veg.pctnonagintrodmanagvegcat,prism_ppt_in,prism_tmin_F,prism_tmean_F,prism_tmax_F,prism_tdmean_F,prism_vpdmin_hPa,prism_vpdmax_hPa,prism_elev_ft
0,HGA,20329198,2017-04-18,33.04581,-117.12715,2017,4,108,Spring,50.0,...,16.61,1.61,0.0,49.8,63.4,77.0,52.9,0.40,22.22,817.0
1,HGA,20329198,2017-04-18,33.04581,-117.12715,2017,4,108,Spring,50.0,...,16.61,1.61,0.0,49.8,63.4,77.0,52.9,0.40,22.22,817.0
2,HGA,20329198,2017-04-18,33.04581,-117.12715,2017,4,108,Spring,492.0,...,16.61,1.61,0.0,49.8,63.4,77.0,52.9,0.40,22.22,817.0
3,HGA,20329198,2017-04-18,33.04581,-117.12715,2017,4,108,Spring,662.0,...,16.61,1.61,0.0,49.8,63.4,77.0,52.9,0.40,22.22,817.0
4,HGA,20329198,2017-04-18,33.04581,-117.12715,2017,4,108,Spring,759.0,...,16.61,1.61,0.0,49.8,63.4,77.0,52.9,0.40,22.22,817.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3111,RBC02,22549103,2022-01-10,33.41544,-117.15199,2022,1,10,Winter,100.0,...,2.50,4.69,0.0,48.8,61.5,74.2,31.0,4.35,23.31,1053.0
3112,RBC04,22549103,2022-01-10,33.41233,-117.15972,2022,1,10,Winter,1790000.0,...,2.50,4.69,0.0,49.2,61.5,73.9,31.6,4.22,22.89,1099.0
3113,RBC04,22549103,2022-01-10,33.41233,-117.15972,2022,1,10,Winter,56.0,...,2.50,4.69,0.0,49.2,61.5,73.9,31.6,4.22,22.89,1099.0
3114,RGT01,22548465,2022-01-10,33.41107,-117.18569,2022,1,10,Winter,1250000.0,...,2.69,2.69,0.0,49.4,61.8,74.2,31.7,4.18,23.13,978.0


In [87]:
full_strct_prism = strct_with_weather[~strct_with_weather["prism_tmean_F"].isna()]

# Base
assert "full_strct_prism" in globals(), "full_strct_prism not found."
df_model = full_strct_prism.copy()

# Keep rows with PRISM (should already be filtered, but harmless)
if "prism_tmean_F" in df_model.columns:
    df_model = df_model[df_model["prism_tmean_F"].notna()].copy()

# Columns to NOT one-hot (keys, coords, targets, timestamp)
protect_cols = {
    "StationCode","COMID","SampleDateTime","Latitude","Longitude",
    "Year","Month","DOY","Result_ugL","Result_ugL_log"
}

# Categorical columns to one-hot (use only those present)
explicit_cat_cols = [
    "Season","ResultQualCode","Analyte","MatrixName","MethodName",
    "SampleTypeCode","CollectionMethodName","PositionWaterColumn","Datum","UnitCollectionDepth",
]
explicit_cat_cols = [c for c in explicit_cat_cols if c in df_model.columns]

print("OHE columns:", explicit_cat_cols)

# Coerce numeric-looking object columns to numeric (prevents accidental OHE)
for c in df_model.select_dtypes(include="object").columns:
    if c in explicit_cat_cols:  # skip intended categoricals
        continue
    s = pd.to_numeric(df_model[c], errors="coerce")
    if s.notna().mean() >= 0.90:
        df_model[c] = s

# One-hot encode the selected categoricals
df_dummies = pd.get_dummies(
    df_model[explicit_cat_cols],
    prefix=explicit_cat_cols,
    drop_first=True,   # avoid perfect collinearity
    dummy_na=False
)

# Join and drop originals
df_model = pd.concat([df_model.drop(columns=explicit_cat_cols), df_dummies], axis=1)

print("df_model shape:", df_model.shape)
print("first 12 cols:", df_model.columns[:12].tolist())


OHE columns: ['Season', 'ResultQualCode', 'Analyte', 'MethodName', 'SampleTypeCode', 'CollectionMethodName', 'PositionWaterColumn', 'Datum', 'UnitCollectionDepth']
df_model shape: (2606, 176)
first 12 cols: ['StationCode', 'COMID', 'SampleDateTime', 'Latitude', 'Longitude', 'Year', 'Month', 'DOY', 'Result_ugL', 'Result_ugL_log', 'CollectionDepth', 'CollectionReplicate']


In [88]:
df_model

,StationCode,COMID,SampleDateTime,Latitude,Longitude,Year,Month,DOY,Result_ugL,Result_ugL_log,...,PositionWaterColumn_Not Recorded,PositionWaterColumn_Subsurface,PositionWaterColumn_Surface,Datum_NAD83,Datum_NAD83,Datum_NR,Datum_WGS 84,Datum_WGS84,Datum_WGS84,UnitCollectionDepth_m
0,HGA,20329198,2017-04-18,33.04581,-117.12715,2017,4,108,50.0,3.931826,...,False,False,False,False,False,True,False,False,False,True
1,HGA,20329198,2017-04-18,33.04581,-117.12715,2017,4,108,50.0,3.931826,...,False,False,False,False,False,True,False,False,False,True
2,HGA,20329198,2017-04-18,33.04581,-117.12715,2017,4,108,492.0,6.200509,...,False,False,False,False,False,True,False,False,False,True
3,HGA,20329198,2017-04-18,33.04581,-117.12715,2017,4,108,662.0,6.496775,...,False,False,False,False,False,True,False,False,False,True
4,HGA,20329198,2017-04-18,33.04581,-117.12715,2017,4,108,759.0,6.633318,...,False,False,False,False,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3111,RBC02,22549103,2022-01-10,33.41544,-117.15199,2022,1,10,100.0,4.615121,...,False,False,False,True,False,False,False,False,False,True
3112,RBC04,22549103,2022-01-10,33.41233,-117.15972,2022,1,10,1790000.0,14.397727,...,False,False,False,True,False,False,False,False,False,True
3113,RBC04,22549103,2022-01-10,33.41233,-117.15972,2022,1,10,56.0,4.043051,...,False,False,False,True,False,False,False,False,False,True
3114,RGT01,22548465,2022-01-10,33.41107,-117.18569,2022,1,10,1250000.0,14.038655,...,False,False,False,True,False,False,False,False,False,True


In [89]:
output_path = '/content/drive/MyDrive/Year 3 Berk/Data Discovery Group Folder'

out_path = f"{output_path}/Data/model_ready.csv"
df_model.to_csv(out_path, index=False)
print("Saved to:", out_path)

Saved to: /content/drive/MyDrive/Year 3 Berk/Data Discovery Group Folder/Data/model_ready.csv


In [ ]:
locs_path = '/content/drive/MyDrive/2025 fall/Data Discovery Group Folder/Data/model_ready.csv'
unique_locs_df = pd.read_csv(locs_path)
unique_locs_df = unique_locs_df.iloc[500:]
unique_locs_df
unique_locs_df.to_csv('remaining_loc.csv', index=False)
files.download('remaining_loc.csv')


In [ ]:
locs_path = '/content/drive/MyDrive/2025 fall/Data Discovery Group Folder/Data/unique_locations (4).csv'
unique_locs_df = pd.read_csv(locs_path)
unique_locs_df = unique_locs_df.iloc[500:]
unique_locs_df
unique_locs_df.to_csv('remaining_loc.csv', index=False)
files.download('remaining_loc.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title
import numpy as np
import pandas as pd

# --- 0) basic null check on keys ---
print("Nulls in strct_smpl keys:",
      strct_smpl[["date_key","lat_key","lon_key"]].isna().any().to_dict())
print("Nulls in prism_ren keys :",
      prism_ren[["date_key","lat_key","lon_key"]].isna().any().to_dict())

# --- 1) where do matches fail? (anti-joins) ---
keys = ["date_key","lat_key","lon_key"]
tmp = strct_smpl.merge(prism_ren[keys].drop_duplicates(), on=keys, how="left", indicator=True)
miss = tmp.loc[tmp["_merge"]=="left_only", keys].copy()
print("Unmatched rows:", len(miss))

# breakup: date-only vs coord-only issues
# (A) Can we match by date+approx coords if we relax rounding?
def try_round(d):
    s = strct_smpl.copy()
    p = prism_ren.copy()
    s["lat_r"] = s["lat_key"].round(d); s["lon_r"] = s["lon_key"].round(d)
    p["lat_r"] = p["lat_key"].round(d); p["lon_r"] = p["lon_key"].round(d)
    m = s.merge(p[["date_key","lat_r","lon_r"]].drop_duplicates(),
                left_on=["date_key","lat_r","lon_r"],
                right_on=["date_key","lat_r","lon_r"],
                how="left", indicator=True)
    return d, (m["_merge"]=="both").sum()

for d in [4,3,2]:
    dgt, cnt = try_round(d)
    print(f"Exact matches if both sides rounded to {dgt} dp:", cnt, f"({cnt/len(strct_smpl):.1%})")

# (B) Date-only join success rate
date_only = strct_smpl.merge(prism_ren[["date_key"]].drop_duplicates(), on="date_key", how="left", indicator=True)
print("Share of rows whose DATE exists in PRISM:", (date_only["_merge"]=="both").mean())

# --- 2) nearest-by-date within ±1 day for currently unmatched (keep coords exact) ---
ss = strct_smpl.copy()
pp = prism_ren.copy()
ss = ss.sort_values("date_key")
pp = pp.sort_values("date_key")
near1 = pd.merge_asof(
    ss, pp[["date_key","lat_key","lon_key"]],
    on="date_key", by=["lat_key","lon_key"],
    tolerance=pd.Timedelta("1D"), direction="nearest"
)
print("Matches within ±1 day:", near1["lat_key_y"].notna().sum(), "of", len(ss))

# --- 3) how far are unmatched rows from the closest PRISM point on the SAME DATE? ---
# build a small candidate window to avoid full cartesian products
u_dates = miss["date_key"].dropna().unique()
cand = prism_ren[prism_ren["date_key"].isin(u_dates)][["date_key","lat_key","lon_key"]].copy()

# join by date, then compute distance in degrees to the nearest prism point that day
win = miss.merge(cand, on="date_key", suffixes=("","_p"))
if not win.empty:
    ddeg = (win["lat_key"]-win["lat_key_p"]).abs() + (win["lon_key"]-win["lon_key_p"]).abs()
    win["ddeg"] = ddeg
    nn = win.sort_values(["date_key","ddeg"]).drop_duplicates(["date_key","lat_key","lon_key"])
    print("Median coord gap (deg) for unmatched on same date:", nn["ddeg"].median())
    print("Share unmatched with nearest ≤ 0.002° (~220 m):", (nn["ddeg"]<=0.002).mean())
else:
    print("No unmatched rows to analyze on same date (unexpected).")

# --- 4) peek a few problematic rows ---
print("\nExamples of unmatched keys:")
display(strct_smpl.loc[tmp["_merge"]=="left_only", ["Longitude","Latitude","SampleDateTime","date_key","lat_key","lon_key"]].head(10))
